# QSC-Quickstart
## Goal of this guide
At the end of this quickstart guide you will have your data fed to the quasiris search cloud, so you can integrate it in your system.

## Register
- Create an account. Visit https://qsc.quasiris.de and click registration.
- Contact us, your account needs to be unlocked.

## Create a basic configuration
- After your account is activated visit https://qsc.quasiris.de and login.
- You need to create an index and a feeding interface. Click on "Quick Start" as shown in the picture below:

![Quick Start](resources/pictures/qsc_home_screen.png)

- You should look at the following screen:

![Quick Start Home](resources/pictures/qsc_quickstart_home.png)

- "Enter code": This is the name of your index. You can choose whatever name you want. It's used to identify the index and other modules realated to this index, as you can create more than one index.
- In the field below you define the data fields you want to make searchable. In our example we are using **"id"**, **"name"**, and **"title"**. Those are formatted as json.
- **Important Note:** If your products have attributs like "color", "wight", "width", "height" etc. don't add them to the field you want to make searchable. Those are handeld seperate. The same goes for categories, if your dataset is categorized.<br>Due to this fact there are some special names you don't want to use as a key:<br>**Fields reserved by the qsc: "attributes" and "categories"**
- So let's enter our fields. As the code we are going to use **"quickstart-index"**:

![Quick Start Data Fields](resources/pictures/qsc_quickstart_enter_data_fields.png)

- Click create search. Next you can specify the language. Don't forget to hit "Save" if you change something before clicking "Next"

![Quick Start General](resources/pictures/qsc_quickstart_general.png)

- In the next screen you can check and edit the type of the data fields. In out case everything is correct. "id", "name" and "title" are all recognized as a string. If you want to make changes click the orange pen on the top right (shown in picture below). When you're done editing click the pen again, save and then next.

![Quick Start General](resources/pictures/qsc_quickstart_edit_data_fields.png)

- The following screens **Ranking** and **Settings** are not relevant to get you started. Simply click next. You can change settings later as you need to.

- At the end you will get a short summary. The most important part is the **API key feeding**. You need this key to feed data to your index, so copy it and save it somewhere safe. Don't worry if you lose this key you can display the key later in the web interface or generate a new one.

![Quick Start Summary](resources/pictures/qsc_quickstart_summary.png)

Now you're all set to start feeding your data to the qsc.

## Feed data to your index
- The very first thing you have to do is to convert your data to the format the qsc is using. The format is described under: [Data format qsc](https://quasiris-search-cloud.com/docs/qsc-admin/admin/feeding/feeding-api-integration#data-structure)<br>If you have trouble converting your data, just contact us. We will write you a converter to convert your data to the format the qsc is using.

## Example
- The example below shows how to feed data to the qsc using the api.

### Resolve all dependencies

In [1]:
# Resolve all dependencies
import json
import re
import getpass
import requests
import time

### To communicate with the API we need some helper functions

In [2]:
def get_json_from_http_post(url: str, post_params: json, headers: dict = None) -> json:
    """
    Requests an API via HTTP post and returns the answer as json object
    :param url: url of the api that should be requested
    :param post_params: params dict to use with e.g. REST APIs
        e.g. {'request': 'search="LED"', 'page': 0, 'size': 100}
    :param headers: dict with additional http headers e.g. auth tokens
    :return: HTTP response parsed into a json object
    """

    # perform a http post, add headers
    response = requests.post(url, post_params, headers=headers)

    # raise Exceptions if any
    response.raise_for_status()

    # return string response as json
    return json.loads(response.text)


def read_file_to_json(filename: str) -> json:
    """
    Reads a json object from a file containing it
    :param filename: relative or full path to the file that contains the json object
    :return: returns the file content as json object
    """

    file = open(filename, mode='r')
    json_object = json.load(file)
    file.close()
    return json_object


def feed_api(url: str, products: json, headers: dict):
    """
    Send list of json objects to api. If an error occurs wait and retry.
    After 5 retries throw exception.
    :param url: url of the api
    :param products: list of json objects
    :param headers: headers to use with api
    """
    
    retries = 5
    wait = 10

    for i in range(retries + 1):
        try:
            get_json_from_http_post(url, json.dumps(products), headers)
            break
        except requests.HTTPError as e:
            if i < retries:
                print(f"Retry in {wait} seconds")
                time.sleep(wait)
            else:
                print("Max retries reached")
                raise e


def convert_icecat_to_qsc(filename: str) -> json:
    """
    Convert products to qsc format
    :param filename: relative path to file
    :return: converted json of products in qsc format
    """
    
    icecat_products = read_file_to_json(filename)
    
    converted_products = []
    
    for product in icecat_products:
        # Set header and payload of product
        tmp = {'header': {"id": product.get("id"), "action": "update"},
               'payload': {"title": product.get("title"), "id": product.get("id"), "name": product.get("name")}
              }
        
        # Create empty list to store attributes
        attributes = []
        
        # Iterate through keys in each product and filter out all keys starting with 'attr'
        for key in product:
            if str(key).startswith("attr"):
                key_tokens = key.split('_')

                # generate attribute name
                attr_name = ' '.join(key_tokens[2:])
                # remove multiple spaces
                attr_name = re.sub(' +', ' ', attr_name)

                # Set datatype of attribute. Valid types using the qsc are string, boolean, long, double, date
                data_type = ""
                
                # string
                if key_tokens[1] == 't':
                    data_type = "string"
                # double
                elif key_tokens[1] == 'n' and isinstance(product.get(key), float):
                    data_type = "double"
                # long
                elif key_tokens[1] == 'n' and isinstance(product.get(key), int):
                    data_type = "long"
                # boolean
                elif key_tokens[1] == 'b':
                    data_type = "boolean"

                # If attribute has a known datatype add type
                if data_type:
                    attributes.append({"name": attr_name, "dataType": data_type, "values": [product.get(key)]})
                # If the datatype is not valid don't add type
                else:
                    attributes.append({"name": attr_name, "values": [product.get(key)]})

        # If attributes were found add them to payload
        if bool(attributes):
            tmp['payload'].update({"attributes": attributes})
            
        # Append converted product to list
        converted_products.append(tmp)
        
    return converted_products

### API credentials
- API URL: The api url has the following format: `https://qsc.quasiris.de/api/v1/data/bulk/qsc/"tenant"/"feeding-code"`<br>
  - Tenant: The tentant is shown on the top right (see picture below). In our case it's **quickstart**<br>
  ![Tenant](resources/pictures/qsc_home_screen_tenant.png)<br>
  - Feeding Code: The feeding-code should be identical to the code you assigned above using the "Quick Start"     option of the qsc. In our case it's **quickstart-index**<br>
  - In our case the API URL is: `https://qsc.quasiris.de/api/v1/data/bulk/qsc/quickstart/quickstart-index`
- API token: The token you got in the summary of the "Quick Start" option above (**API key feeding**).

In [ ]:
# Read parameters

# Get tenant
tenant = input("Tenant:")

# Get feeding code
feeding_code = input("Feeding Code:")

# Get api token
token = getpass.getpass("API token:")

# Define base url
base_url = "https://qsc.quasiris.de/api/v1/data/bulk/qsc/"
# Use dev environment
# base_url = "https://qsc-dev.quasiris.de/api/v1/data/bulk/qsc/"

# Generate url
url = base_url + tenant + "/" + feeding_code

# Generate headers
headers = {"Content-type": "application/json", "X-QSC-Token": token}

# Print api url
print(f"API URL is: {url}")

### Load products and push to API

In [4]:
# The data that is pushed to the qsc api is stored in 'products'.
# Load products and convert them to the qsc format.
# If you want to use your data replace the 'convert_icecat_to_qsc' function
# with your function to convert your data to the qsc format.
products = convert_icecat_to_qsc("resources/350-sample-products.json")

# Send 100 products in one batch to api
batch_size = 100

# Helper variables
batch = []
product_counter = 0
batch_counter = 0

# Iterate through products and feed batch wise
for product in products:
    batch.append(product)
    product_counter += 1
    
    # If batch size is reached send batch to api and reset batch
    if product_counter == batch_size:
        time.sleep(1)
        feed_api(url, batch, headers)
        batch_counter += 1
        print(f"Batch number {batch_counter} sent!")
        batch.clear()
        product_counter = 0

# Send remaining products
if bool(batch):
    time.sleep(1)
    feed_api(url, batch, headers)
    print(f"Remaining {product_counter} products sent!")

# Print number of products sent to api
print(f"A total of {((batch_counter * batch_size) + product_counter)} products were sent to api!")

Batch number 1 sent!
Batch number 2 sent!
Batch number 3 sent!
Remaining 50 products sent!
A total of 350 products were sent to api!


## Check if products are available in index
- Go to `Search->"your code":` Click "Search" tab. On the top right you should see how much products your index is holding.
- **Note:** We pushed 35**0** products to the api but our index is holding 35**1** products.<br>
That's because we used the example data at the beginning when using the "Quick Start" guide to create a product and initialize some things. You can delete that product if you want.
![Search](resources/pictures/qsc_search.png)